# Book Rating Predictor
### PolyU COMP5434 Term Project


In [1]:
#Load data
import pandas as pd

dataFrame = pd.read_csv('data/Train_data.csv', error_bad_lines = False)

#observe the first 5 data frames
dataFrame.head() 
#summary of the data frames
# dataFrame.describe()
#check missing values
# dataFrame.isnull().sum()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,8253,Little Town on the Prairie (Little House #7),Laura Ingalls Wilder/Garth Williams,4.19,0060885432,9780060885434,eng,374,71853,646,1/1/2007,HarperTrophy
1,11101,First Comes Love (Hot Water California #1),Christie Ridgway,3.81,0380818957,9780380818952,en-US,384,1051,47,1/1/2002,Avon Books
2,28193,When Santa Fell to Earth,Cornelia Funke/Paul Howard/Oliver G. Latsch,3.76,043978204X,9780439782043,eng,167,1840,245,10/1/2006,Chicken House / Scholastic
3,25047,The Case of the Stinky Science Project (Jigsaw...,James Preller/John Speirs/James Preller,3.83,0439114284,9780439114288,en-US,80,162,11,3/1/2001,Scholastic Paperbacks
4,17800,Dance Dance Dance,Haruki Murakami,4.04,0099448769,9780099448761,eng,393,50169,2117,2/7/2002,Vintage


In [2]:
#Data preprocessing
from sklearn.preprocessing import OrdinalEncoder

encoding = {'language_code':{'en-US': 'eng', 'en-GB': 'eng', 'en-CA': 'eng'}} # Unify the langauge codes
dataFrame.replace(encoding, inplace=True)

enc = OrdinalEncoder()
enc.fit(dataFrame[['language_code']])
dataFrame[['language_code']] = enc.fit_transform(dataFrame[['language_code']]) # Apply ordinal encoding on language_code to convert it into numerical column

In [3]:
#1167609600  1970-01-01 00:00:00 to 2007-01-01
#1167580800 1970-01-01  08:00:00 to 2007-01-01
#The map-reduce algorithm to convert date string to timestamp
dataFrame.loc[dataFrame.bookID == 31373, 'publication_date'] = '10/01/1999'
dataFrame.loc[dataFrame.bookID == 45531, 'publication_date'] = '10/01/1975'
import time
import datetime
#The map function returns like 1/1/2007,2007-07-01 00:00:00

# mm,dd,yyyy = dataFrame['publication_date'][1].split('/')
# print(dataFrame['publication_date'][1])


def map():
    kv=[]
    for i in range(0, len(dataFrame)):
        mm,dd,yyyy = dataFrame['publication_date'][i].split('/')
        date = dataFrame['publication_date'][i]
        value = yyyy+"-"+mm+"-"+dd+" "+"00:00:00"
        kv.append([date, value])
#         print(",".join([date, value]))
#     print(kv)
    return kv

#only works for later than 1970
# def reduce(kv):
#     for i in range(0, len(kv)):
#         key = kv[i][0]
#         date_and_time = time.strptime(kv[i][1], "%Y-%m-%d %H:%M:%S")
#         value = int(time.mktime(date_and_time))
#         print(value)


#works for before 1970 and later
def reduce(kv):
    value=[]
    for i in range(0, len(kv)):
        key = kv[i][0]
        date_and_time = datetime.datetime.strptime(kv[i][1], "%Y-%m-%d %H:%M:%S")
        start_time = datetime.datetime.strptime("1970-01-01 00:00:00","%Y-%m-%d %H:%M:%S" )
        difference_time = date_and_time - start_time
#         print(difference_time)
#         print(difference_time.seconds)
        value_i = difference_time.days*24*3600 + difference_time.seconds
        value.append(value_i)
    return value
kv=map()

timestamp = reduce(kv)


In [4]:
print(timestamp)

[1167609600, 1009843200, 1159660800, 983404800, 1013040000, 1162425600, 854409600, 1162252800, 1083801600, 844128000, 1140739200, 809913600, 1093996800, 373161600, 1138752000, 1054598400, 1128124800, 1127779200, 423273600, 1114387200, 1078790400, 1128384000, 1091750400, 920246400, 1230768000, 1128124800, 1054425600, 863136000, 229132800, 829526400, 1130803200, 1041897600, 683683200, 1171843200, 1007164800, 1139875200, 1016064000, 1077580800, 979776000, 854755200, 1046476800, 1120176000, 1104537600, 1126742400, 1168300800, 1139875200, 455846400, 668822400, 778377600, 1072742400, 862444800, 1115078400, 1113696000, 1000944000, 746841600, 959817600, 1139270400, 1109635200, 1079308800, 1160524800, 1101859200, 1014940800, 720144000, 888710400, 1046131200, -460339200, 651801600, 967766400, 1164672000, 1095984000, 1159833600, 1071360000, 1155600000, 1041379200, 877219200, 746841600, 1127088000, 532137600, 297043200, 1099872000, 953596800, 854496000, 1074556800, 454896000, 1017878400, 115188480

In [5]:
dataFrame['timestamp']= timestamp
print(dataFrame['timestamp'])

0       1167609600
1       1009843200
2       1159660800
3        983404800
4       1013040000
5       1162425600
6        854409600
7       1162252800
8       1083801600
9        844128000
10      1140739200
11       809913600
12      1093996800
13       373161600
14      1138752000
15      1054598400
16      1128124800
17      1127779200
18       423273600
19      1114387200
20      1078790400
21      1128384000
22      1091750400
23       920246400
24      1230768000
25      1128124800
26      1054425600
27       863136000
28       229132800
29       829526400
           ...    
9970     488592000
9971    1170720000
9972    1162425600
9973    1099353600
9974    1148774400
9975     816307200
9976     197164800
9977    1023667200
9978     967420800
9979    1159228800
9980    1070323200
9981    1047945600
9982    1046131200
9983     983059200
9984     441763200
9985    1069286400
9986     712627200
9987     909446400
9988    1094515200
9989     962582400
9990    1014854400
9991     951

In [27]:
dataFrame['publication_date'] = pd.to_datetime(dataFrame['publication_date'], format='%m/%d/%Y', errors='coerce') # Convert data type of publication_date from object into date type
dataFrame[dataFrame['publication_date'].isnull()]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher


In [37]:
dataFrame.loc[dataFrame.bookID == 31373, 'publication_date'] = '1999-10-01 00:00:00'
dataFrame.loc[dataFrame.bookID == 45531, 'publication_date'] = '1975-10-01 00:00:00'

In [6]:
dataFrame['year'] = pd.DatetimeIndex(dataFrame['publication_date']).year # Extract year of publication in a separate column

dataFrame.rename(columns = {'  num_pages': 'num_pages'}, inplace=True) # Rename the column to remove leading whitespaces

In [7]:
dataFrame['authors_occ'] = dataFrame.groupby('authors')['authors'].transform('count') # Add a new feature which has the number of occurences of each book

In [8]:
dataFrame.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,year,authors_occ
0,8253,Little Town on the Prairie (Little House #7),Laura Ingalls Wilder/Garth Williams,4.19,0060885432,9780060885434,1.0,374,71853,646,2007-01-01 00:00:00,HarperTrophy,2007,6
1,11101,First Comes Love (Hot Water California #1),Christie Ridgway,3.81,0380818957,9780380818952,1.0,384,1051,47,2002-01-01 00:00:00,Avon Books,2002,1
2,28193,When Santa Fell to Earth,Cornelia Funke/Paul Howard/Oliver G. Latsch,3.76,043978204X,9780439782043,1.0,167,1840,245,2006-10-01 00:00:00,Chicken House / Scholastic,2006,1
3,25047,The Case of the Stinky Science Project (Jigsaw...,James Preller/John Speirs/James Preller,3.83,0439114284,9780439114288,1.0,80,162,11,2001-03-01 00:00:00,Scholastic Paperbacks,2001,1
4,17800,Dance Dance Dance,Haruki Murakami,4.04,0099448769,9780099448761,1.0,393,50169,2117,2002-02-07 00:00:00,Vintage,2002,2


In [76]:
dataFrame['num_occ'] = dataFrame.groupby('title')['title'].transform('count') # Add a new feature which has the number of occurences of each book

In [77]:
dataFrame.head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,year,authors_occ,num_occ
0,8253,Little Town on the Prairie (Little House #7),Laura Ingalls Wilder/Garth Williams,4.19,0060885432,9780060885434,1.0,374,71853,646,2007-01-01 00:00:00,HarperTrophy,2007,6,1
1,11101,First Comes Love (Hot Water California #1),Christie Ridgway,3.81,0380818957,9780380818952,1.0,384,1051,47,2002-01-01 00:00:00,Avon Books,2002,1,1
2,28193,When Santa Fell to Earth,Cornelia Funke/Paul Howard/Oliver G. Latsch,3.76,043978204X,9780439782043,1.0,167,1840,245,2006-10-01 00:00:00,Chicken House / Scholastic,2006,1,1
3,25047,The Case of the Stinky Science Project (Jigsaw...,James Preller/John Speirs/James Preller,3.83,0439114284,9780439114288,1.0,80,162,11,2001-03-01 00:00:00,Scholastic Paperbacks,2001,1,1
4,17800,Dance Dance Dance,Haruki Murakami,4.04,0099448769,9780099448761,1.0,393,50169,2117,2002-02-07 00:00:00,Vintage,2002,2,1
5,23601,The J.R.R. Tolkien Companion and Guide Volume...,Wayne G. Hammond/Wayne G. Hammond,4.46,0618391010,9780618391011,1.0,1262,31,2,2006-11-02 00:00:00,Houghton Mifflin Harcourt,2006,1,1
6,92,Mapping the Big Picture: Integrating Curriculu...,Heidi Hayes Jacobs,3.68,0871202867,9780871202864,1.0,108,77,2,1997-01-28 00:00:00,Association for Supervision & Curriculum Devel...,1997,3,1
7,39501,The Burglar Who Thought He Was Bogart (Bernie ...,Lawrence Block,3.89,0060872799,9780060872793,1.0,400,2134,99,2006-10-31 00:00:00,HarperTorch,2006,7,1
8,37387,The Prime of Miss Jean Brodie / The Girls of S...,Muriel Spark,3.79,1857152743,9781857152746,1.0,512,1477,90,2004-05-06 00:00:00,Random House,2004,2,2
9,32715,Desperation / The Regulators: Box Set,Stephen King/Richard Bachman,4.07,067077605X,9780670776054,1.0,1075,3231,72,1996-10-01 00:00:00,Viking Adult,1996,2,1


In [78]:
dataFrame['rate_occ'] = dataFrame['average_rating'] * dataFrame['num_occ']
dataFrame['rate_weight'] = dataFrame['average_rating'] * dataFrame['text_reviews_count']
dataFrame['rate_weight_2'] = dataFrame['average_rating'] * dataFrame['ratings_count']
dataFrame['rate_per_pages'] = dataFrame['average_rating'] * dataFrame['num_pages']

In [1]:
dataFrame.head()

NameError: name 'dataFrame' is not defined

In [79]:
#Creating model
label = dataFrame['average_rating'].values
dataFrame.drop(['bookID', 'title', 'authors', 'isbn', 'isbn13', 'publication_date', 'publisher', 'average_rating'], axis=1, inplace=True)

In [80]:
dataFrame.head()

,language_code,num_pages,ratings_count,text_reviews_count,year,authors_occ,num_occ,rate_occ,rate_weight,rate_weight_2,rate_per_pages
0,1.0,374,71853,646,2007,6,1,4.19,2706.74,301064.07,1567.06
1,1.0,384,1051,47,2002,1,1,3.81,179.07,4004.31,1463.04
2,1.0,167,1840,245,2006,1,1,3.76,921.20,6918.40,627.92
3,1.0,80,162,11,2001,1,1,3.83,42.13,620.46,306.40
4,1.0,393,50169,2117,2002,2,1,4.04,8552.68,202682.76,1587.72


In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataFrame, label, test_size=0.3)

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4))

parameters = {
    'learning_rate': [0.001, 0.01, 0.02, 0.1, 0.2, 1.0],
    'n_estimators': [10, 50, 100, 200]
}

grad_Ada = GridSearchCV(model, parameters, refit=True)
grad_Ada.fit(X_train, y_train)

print('Best Score: ', grad_Ada.best_score_*100, '\nBest Parameters: ', grad_Ada.best_params_)

D:\winPython\WPy64-3740\python-3.7.4.amd64\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best Score:  94.10393767465519 
Best Parameters:  {'learning_rate': 0.1, 'n_estimators': 100}


In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

model  = LinearRegression()

parameters = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    
}

grad_Linear = GridSearchCV(model, parameters, refit=True)
grad_Linear.fit(X_train, y_train)

print('Best Score: ', grad_Linear.best_score_*100, '\nBest Parameters: ', grad_Linear.best_params_)

Best Score:  83.97994712131228 
Best Parameters:  {'fit_intercept': True, 'normalize': False}


D:\winPython\WPy64-3740\python-3.7.4.amd64\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [79]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
pred_lr = grad_Linear.predict(X_test)

# Check Model Score
print("Residual sum of squares: ",  np.mean((pred_lr - y_test) ** 2))
print('MSE:', metrics.mean_squared_error(y_test, pred_lr))#均方误差
print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, pred_lr))))
print('Model Score on Test Data: ', grad_Linear.score(pred_lr, y_test))

NameError: name 'grad_Linear' is not defined

In [18]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10, 12, 15],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [5, 10, 15]
}

grad_rf = GridSearchCV(model, parameters, refit=True, cv=10)
grad_rf.fit(X_train, y_train)

print('Best Score: ', grad_rf.best_score_*100, '\nBest Parameters: ', grad_rf.best_params_)

KeyboardInterrupt: 